# Load Dataset

In [1]:
!pip install -q datasets
!pip install dahuffman
!pip install transformers

In [2]:
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import ViTModel
import numpy as np
import torch
import cv2
import torch.nn as nn
from transformers import ViTModel, ViTConfig
from torchvision import transforms
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import torch.nn as nn
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import numpy as np
from collections import Counter
import torch
from collections import Counter
from dahuffman import HuffmanCodec



In [3]:
dataset = load_dataset("Matthijs/snacks")
print(dataset)
print(dataset["train"].features['label'].names)

labels = dataset["train"].features["label"].names
num_labels = len(dataset["train"].features["label"].names)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
print(label2id)
print(id2label)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4838
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 952
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 955
    })
})
['apple', 'banana', 'cake', 'candy', 'carrot', 'cookie', 'doughnut', 'grape', 'hot dog', 'ice cream', 'juice', 'muffin', 'orange', 'pineapple', 'popcorn', 'pretzel', 'salad', 'strawberry', 'waffle', 'watermelon']
{'apple': 0, 'banana': 1, 'cake': 2, 'candy': 3, 'carrot': 4, 'cookie': 5, 'doughnut': 6, 'grape': 7, 'hot dog': 8, 'ice cream': 9, 'juice': 10, 'muffin': 11, 'orange': 12, 'pineapple': 13, 'popcorn': 14, 'pretzel': 15, 'salad': 16, 'strawberry': 17, 'waffle': 18, 'watermelon': 19}
{0: 'apple', 1: 'banana', 2: 'cake', 3: 'candy', 4: 'carrot', 5: 'cookie', 6: 'doughnut', 7: 'grape', 8: 'hot dog', 9: 'ice cream', 10: 'juice', 11: 'muffin', 12: 'orange', 13: 'pineapple', 14: 'popcorn', 15: 'pretzel'

# Forward Pass ViT Model with a Toy Image

In [4]:
# Create toy image with dim (batch x channel x width x height)
toy_img = torch.rand(1, 3, 48, 48)

# Define conv layer parameters
num_channels = 3
hidden_size = 768
patch_size = 16

projection = nn.Conv2d(num_channels, hidden_size, kernel_size=patch_size,
             stride=patch_size)

out_projection = projection(toy_img)

print(f'Original image size: {toy_img.size()}')
print(f'Size after projection: {out_projection.size()}')


patch_embeddings = out_projection.flatten(2).transpose(1, 2)
print(f'Patch embedding size: {patch_embeddings.size()}')

# Define [CLS] token embedding with the same emb dimension as the patches
batch_size = 1
cls_token = nn.Parameter(torch.randn(1, 1, hidden_size))
cls_tokens = cls_token.expand(batch_size, -1, -1)

# Prepend [CLS] token in the beginning of patch embedding
patch_embeddings = torch.cat((cls_tokens, patch_embeddings), dim=1)
print(f'Patch embedding size: {patch_embeddings.size()}')

# Define position embedding with the same dimension as the patch embedding
position_embeddings = nn.Parameter(torch.randn(batch_size, 10, hidden_size))

# Add position embedding into patch embedding
input_embeddings = patch_embeddings + position_embeddings
print(f'Input embedding size: {input_embeddings.size()}')


# Define parameters for ViT-base (example)
num_heads = 12
num_layers = 12

# Define Transformer encoders' stack
transformer_encoder_layer = nn.TransformerEncoderLayer(
           d_model=hidden_size, nhead=num_heads,
           dim_feedforward=int(hidden_size * 4),
           dropout=0.1)
transformer_encoder = nn.TransformerEncoder(
           encoder_layer=transformer_encoder_layer,
           num_layers=num_layers)

# Forward pass
output_embeddings = transformer_encoder(input_embeddings)
print(f' Output embedding size: {output_embeddings.size()}')

# Load pretrained model
model_checkpoint = 'google/vit-base-patch16-224-in21k'
model = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)

# Example input image
input_img = torch.rand(batch_size, num_channels, 224, 224)

# Forward pass input image
output_embedding = model(input_img)
print(output_embedding)
print(f"Ouput embedding size: {output_embedding['last_hidden_state'].size()}")

num_labels = 20

# Define linear classifier layer
classifier = nn.Linear(hidden_size, num_labels)

# Forward pass on the output embedding of [CLS] token
output_classification = classifier(output_embedding['last_hidden_state'][:, 0, :])
print(f"Output embedding size: {output_classification.size()}")

Original image size: torch.Size([1, 3, 48, 48])
Size after projection: torch.Size([1, 768, 3, 3])
Patch embedding size: torch.Size([1, 9, 768])
Patch embedding size: torch.Size([1, 10, 768])
Input embedding size: torch.Size([1, 10, 768])
 Output embedding size: torch.Size([1, 10, 768])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.0465, -0.1856,  0.1091,  ...,  0.2508,  0.0281, -0.3495],
         [ 0.2315,  0.0414,  0.0920,  ..., -0.0346,  0.1361, -0.2402],
         [-0.0076, -0.3808,  0.0396,  ...,  0.1155,  0.1496, -0.3235],
         ...,
         [ 0.1400, -0.1947,  0.0199,  ..., -0.0722,  0.2207, -0.3628],
         [ 0.0037, -0.1514, -0.1328,  ...,  0.0537,  0.0810, -0.2906],
         [ 0.1674, -0.0480,  0.0814,  ...,  0.0199,  0.0538, -0.2646]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=None, hidden_states=None, attentions=None)
Ouput embedding size: torch.Size([1, 197, 768])
Output embedding size: torch.Size([1, 20])


# Fine Tuning ViT Impelementation

In [5]:
#Pretrained model checkpoint
model_checkpoint = 'google/vit-base-patch16-224-in21k'

In [6]:
class ImageDataset(torch.utils.data.Dataset):

  def __init__(self, input_data):

      self.input_data = input_data
      self.transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224), antialias=True),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
        ])

  def __len__(self):
      return len(self.input_data)

  def get_images(self, idx):
      return self.transform(self.input_data[idx]['image'])

  def get_labels(self, idx):
      return self.input_data[idx]['label']

  def __getitem__(self, idx):
      # Get input data in a batch
      train_images = self.get_images(idx)
      train_labels = self.get_labels(idx)

      return train_images, train_labels

In [7]:
def train_model(model, train_dataset, epochs, learning_rate, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = model.to(device)

    train_loader = DataLoader(ImageDataset(train_dataset), batch_size=batch_size, shuffle=True, num_workers=1)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        total_loss, total_acc = 0, 0

        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

        avg_loss = total_loss / len(train_loader.dataset)
        avg_acc = total_acc / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return model

def test_model(model, test_dataset, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = model.to(device)

    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model.eval()
    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc

In [8]:
import torch
import torch.nn as nn
from transformers import ViTModel, ViTConfig

class ViT(nn.Module):
    def __init__(self, config=ViTConfig(), num_labels=20,
                 model_checkpoint='google/vit-base-patch16-224-in21k',
                 encoded_weights=None, codec=None):
        super(ViT, self).__init__()
        self.vit = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.encoded_weights = encoded_weights
        self.codec = codec

    def forward(self, x):
        x = self.vit(x)['last_hidden_state']
        output = self.classifier(x[:, 0, :])
        return output


In [14]:
# Hyperparameters
EPOCHS = 3
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")



In [ ]:
model = ViT().to(device)
trained_model = train_model(model, dataset['train'], EPOCHS, LEARNING_RATE, BATCH_SIZE)

In [15]:
# load weights if you have it locally so don't have to fine tune again

trained_model =  ViT()
trained_model.load_state_dict(torch.load('original_model.pth'))

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [16]:
test_loss, test_accuracy = test_model(trained_model, dataset['test'], BATCH_SIZE)
print(test_loss, test_accuracy)

100%|██████████| 30/30 [00:10<00:00,  2.95it/s]

Test - Loss: 0.0096, Accuracy: 0.9191
0.009641023243174833 0.9191176470588235


In [17]:
ViT()

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ViT(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072,

# Round model weights

In [48]:
import copy


trained_model.eval()  # Set the model to evaluation mode
model.to('cpu')  # Move the model to CPU

rounded_model = copy.deepcopy(trained_model)
rounded_model.to('cpu')
precision = 2

def round_with_decimal_precision(tensor, decimal_precision=2):
    # Convert the PyTorch tensor to a NumPy array
    numpy_array = tensor.numpy()

    # Apply rounding with decimal precision using NumPy
    rounded_numpy_array = np.round(numpy_array, decimals=decimal_precision)

    # Convert the rounded NumPy array back to a PyTorch tensor
    rounded_tensor = torch.tensor(rounded_numpy_array, dtype=tensor.dtype)

    return rounded_tensor




for name, param in rounded_model.named_parameters():
    param.data = round_with_decimal_precision(param.data, decimal_precision=precision).to(torch.float16)
    print(param)
    print(param.dtype)


Streaming output truncated to the last 5000 lines.
Parameter containing:
tensor([ 0.1300,  0.1801, -0.1000,  0.0300, -0.0100, -0.0200,  0.0200, -0.1100,
         0.0200, -0.1400, -0.0100,  0.0000, -0.0400,  0.0200, -0.0400, -0.1100,
         0.0800,  0.0300,  0.0900,  0.0700, -0.0400,  0.0800, -0.0000, -0.0900,
        -0.0500,  0.0400, -0.0300,  0.0100,  0.0300, -0.1000,  0.0700, -0.0100,
        -0.1000, -0.1100, -0.0800,  0.0800,  0.1000,  0.1100, -0.0400, -0.0500,
         0.0400,  0.0200,  0.0400,  0.1000,  0.0800,  0.0200, -0.1500, -0.1200,
         0.0200,  0.0300,  0.0500,  0.2500, -0.0600, -0.0100,  0.0300,  0.0100,
        -0.0700, -0.0100, -0.0500,  0.0400,  0.0600,  0.0500,  0.0200,  0.1600,
         0.0700, -0.0200, -0.0500, -0.0200,  0.0200,  0.1000, -0.0600,  0.0000,
        -0.0300,  0.0300,  0.0800, -0.0400,  0.0100, -0.1000, -0.0600, -0.0500,
        -0.0400,  0.0500,  0.0000,  0.0500,  0.0000,  0.1000, -0.2800,  0.0200,
        -0.0100, -0.0500,  0.0900, -0.0900, -0.

In [49]:
print("Original Weights:")
for param in trained_model.parameters():
    print(param.data)
    print(param.dtype)

Streaming output truncated to the last 5000 lines.
        -5.7620e-02, -5.1544e-02,  5.4078e-02,  1.9280e-02,  4.1263e-03,
        -2.3673e-02,  3.9147e-01, -6.9390e-03, -4.6367e-02,  3.5841e-02,
        -5.7185e-02,  9.1228e-02, -1.4987e-02, -6.4177e-02, -6.5082e-02,
        -1.5991e-03, -6.4435e-02, -4.7444e-02,  2.9985e-02,  6.9871e-02,
         1.7889e-02, -3.1288e-03, -2.5162e-02, -2.8510e-02, -5.1084e-02,
        -5.5079e-02,  8.3184e-03,  1.0842e-02, -4.1202e-02,  2.6546e-02,
         4.5583e-02, -3.0112e-02,  1.8328e-02, -1.0709e-02, -3.1318e-02,
        -1.2585e-02,  3.5383e-02,  2.1501e-02, -5.3763e-02, -3.7328e-02,
         1.9015e-02,  6.7549e-01, -5.0134e-02, -4.4700e-03,  2.3751e-02,
        -2.3151e-03, -6.4195e-02, -4.1891e-02,  8.8386e-02,  3.4226e-02,
        -4.3641e-02, -2.2520e-02,  5.3559e-02,  2.0353e-02, -2.5555e-02,
         5.1863e-02,  3.5027e-02,  3.9888e-02, -5.5932e-02,  4.8187e-02,
        -3.1513e-02, -1.4611e-02,  6.6252e-03, -3.5203e-03,  2.7612e-02,


In [50]:
def test_model_rounded(model, test_dataset, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model.eval()
    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc

# Test the model
test_loss, test_accuracy = test_model_rounded(rounded_model, dataset['test'], BATCH_SIZE)
print(test_loss, test_accuracy)

100%|██████████| 30/30 [00:04<00:00,  7.26it/s]

Test - Loss: 0.0097, Accuracy: 0.9202
0.00972382361147584 0.9201680672268907


In [51]:
import torch
import os
model.to("cpu")
# Assuming 'model' is your original ViT model and 'quantized_model' is the quantized version

# Save the original model
torch.save(trained_model.state_dict(), 'original_model.pth')

# Save the quantized model
torch.save(rounded_model.state_dict(), 'rounded_model.pth')

# Function to get file size in MB
def file_size_in_mb(file_path):
    size_bytes = os.path.getsize(file_path)
    return size_bytes / (1024 * 1024)  # Convert bytes to megabytes

# Print the file sizes
original_size = file_size_in_mb('original_model.pth')
rounded_model_size = file_size_in_mb('rounded_model.pth')
print(f"Original Model Size: {original_size:.2f} MB")
print(f"Rounded Model Size: {rounded_model_size:.2f} MB")

Original Model Size: 327.43 MB
Rounded Model Size: 163.76 MB


# Huffman encoding

In [52]:
# Collect encoding statistics


device = torch.device("cpu")
rounded_model.to(device)


all_weights_flat = []


for name, param in rounded_model.named_parameters():
    layer_weights = param.data.cpu().numpy().flatten().tolist()
    all_weights_flat.extend(layer_weights)


frequencies = Counter(all_weights_flat)
codec = HuffmanCodec.from_frequencies(frequencies)

print(frequencies)

Counter({0.0: 4801449, -0.01000213623046875: 4717356, 0.01000213623046875: 4716741, -0.0200042724609375: 4517700, 0.0200042724609375: 4512296, -0.029998779296875: 4232798, 0.029998779296875: 4217589, -0.040008544921875: 3888358, 0.040008544921875: 3881114, -0.04998779296875: 3508670, 0.04998779296875: 3503996, -0.05999755859375: 3125535, 0.05999755859375: 3115117, -0.07000732421875: 2742056, 0.07000732421875: 2738387, -0.08001708984375: 2375592, 0.08001708984375: 2368858, -0.09002685546875: 2028569, 0.09002685546875: 2028544, -0.0999755859375: 1716013, 0.0999755859375: 1714547, 0.1099853515625: 1437137, -0.1099853515625: 1437070, 0.1199951171875: 1192730, -0.1199951171875: 1189565, 0.1300048828125: 981643, -0.1300048828125: 976704, 0.1400146484375: 801432, -0.1400146484375: 797911, 0.1500244140625: 650192, -0.1500244140625: 645079, 0.1600341796875: 523162, -0.1600341796875: 519036, 0.1700439453125: 420296, -0.1700439453125: 415243, 0.1800537109375: 334314, -0.1800537109375: 332545, 0.1

In [56]:
huffman_model = copy.deepcopy(rounded_model)
encoded_weights_dict = {}



for name, param in huffman_model.named_parameters():
    layer_weights = param.data.flatten().tolist()
    encoded_weights = codec.encode(layer_weights)
    encoded_weights_dict[name] = encoded_weights
    decoded_weights = codec.decode(encoded_weights)
    assert layer_weights == decoded_weights, "Mismatch in encoding/decoding!"

torch.save(encoded_weights_dict, 'encoded_model_weights.pth')

In [57]:

huffman_size = file_size_in_mb('encoded_model_weights.pth')
print(f"Huffman Model Size: {huffman_size:.2f} MB")

Huffman Model Size: 80.13 MB


# Test model

In [58]:
def test_model_huffman(model, test_dataset, batch_size, codec, encoded_weights_file):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model = model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc, model.state_dict()

## Test entire decoded Huffman model

In [59]:
encoded_weights_dict = torch.load('encoded_model_weights.pth')
model = ViT()
model.to(device)

for name, param in model.named_parameters():
    if name in encoded_weights_dict:
        decoded_weights = codec.decode(encoded_weights_dict[name])
        decoded_weights_tensor = torch.tensor(decoded_weights, device=device).view_as(param.data)
        param.data.copy_(decoded_weights_tensor)
    else:
        print(f"Weight for layer {name} not found in encoded weights.")

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [60]:
model.eval()
test_loss, test_accuracy, state_dic = test_model_huffman(model, dataset['test'], BATCH_SIZE, codec, 'encoded_model_weights.pth')
print(test_loss, test_accuracy)

100%|██████████| 30/30 [00:10<00:00,  2.85it/s]

Test - Loss: 0.0097, Accuracy: 0.9202
0.009724996514430567 0.9201680672268907


## Test dynamically decoded huffman model